In [46]:
from pyspark.sql import SparkSession
import pandas as pd
from io import StringIO

spark=SparkSession.builder.appName("Colab PySpark Setup").getOrCreate()

spark


In [47]:
#1. Create a new database named sales_db.
spark.sql("""create database if not exists sales_db""")
#2. Set the current database to sales_db.
spark.sql("""use sales_db""")
#3. Create a table product_sales with columns:
#"""ProductID (INT)
# ProductName (STRING)
# Category (STRING)
# Price (DOUBLE)
# Quantity (INT)
# SaleDate (DATE)
#"""
spark.sql("""create table if not exists product_sales(productid int,
productname string, category string,
price double, quantity int, saledate date) using parquet""")
#4. Insert at least 5 rows into product_sales.
spark.sql("""insert into product_sales values
  (201, 'Smartphone', 'Electronics', 45000.00, 3, date '2024-06-01'),
  (202, 'Office Chair', 'Furniture', 3000.00, 2, date '2024-06-02'),
  (203, 'Notebook', 'Stationery', 50.00, 10, date '2024-06-03'),
  (204, 'Headphones', 'Electronics', 1500.00, 4, date '2024-06-04'),
  (205, 'Water Bottle', 'Accessories', 250.00, 6, date '2024-06-05');
""")


DataFrame[]

In [48]:
#5. Select all records from product_sales.
spark.sql("""select * from product_sales""").show()
#6. Retrieve products where price is above 500.
spark.sql("""select * from product_sales where Price > 500""").show()
#7. Calculate total sale amount (Price * Quantity) for each product.
spark.sql("""select ProductName, Price * Quantity as TotalSaleAmount from product_sales""").show()
#8. Find the number of products sold in each Category.
spark.sql("""select Category, count(ProductID) as NumberOfProductsSold from product_sales group by Category""").show()
#9. Sort products by total sales in descending order.
spark.sql("""select ProductName, Price * Quantity as TotalSaleAmount from product_sales order by TotalSaleAmount desc""").show()


+---------+------------+-----------+-------+--------+----------+
|productid| productname|   category|  price|quantity|  saledate|
+---------+------------+-----------+-------+--------+----------+
|      203|    Notebook| Stationery|   50.0|      10|2024-06-03|
|      204|  Headphones|Electronics| 1500.0|       4|2024-06-04|
|      205|Water Bottle|Accessories|  250.0|       6|2024-06-05|
|      201|  Smartphone|Electronics|45000.0|       3|2024-06-01|
|      202|Office Chair|  Furniture| 3000.0|       2|2024-06-02|
+---------+------------+-----------+-------+--------+----------+

+---------+------------+-----------+-------+--------+----------+
|productid| productname|   category|  price|quantity|  saledate|
+---------+------------+-----------+-------+--------+----------+
|      204|  Headphones|Electronics| 1500.0|       4|2024-06-04|
|      201|  Smartphone|Electronics|45000.0|       3|2024-06-01|
|      202|Office Chair|  Furniture| 3000.0|       2|2024-06-02|
+---------+------------+

In [49]:
#10. Create a PySpark DataFrame with dummy product data.
from pyspark.sql import Row
data = [
    Row(206, 'Laptop', 'Electronics', 75000.0, 2, '2024-06-06'),
    Row(207, 'Bookshelf', 'Furniture', 5200.0, 1, '2024-06-07'),
    Row(208, 'Pen Set', 'Stationery', 120.0, 15, '2024-06-08'),
    Row(209, 'Bluetooth Speaker', 'Electronics', 3200.0, 3, '2024-06-09'),
    Row(210, 'Coffee Mug', 'Accessories', 350.0, 5, '2024-06-10'),
]
# Create DataFrame
df = spark.createDataFrame(data, ["ProductID", "ProductName", "Category", "Price", "Quantity", "SaleDate"])

#11. Register it as a temporary view called temp_orders.
# Register temporary view
df.createOrReplaceTempView("temp_orders")

#12. Run a SQL query to filter temp_orders where quantity > 1.
spark.sql("select * from product_sales where quantity>1").show()

+---------+------------+-----------+-------+--------+----------+
|productid| productname|   category|  price|quantity|  saledate|
+---------+------------+-----------+-------+--------+----------+
|      203|    Notebook| Stationery|   50.0|      10|2024-06-03|
|      204|  Headphones|Electronics| 1500.0|       4|2024-06-04|
|      205|Water Bottle|Accessories|  250.0|       6|2024-06-05|
|      201|  Smartphone|Electronics|45000.0|       3|2024-06-01|
|      202|Office Chair|  Furniture| 3000.0|       2|2024-06-02|
+---------+------------+-----------+-------+--------+----------+



In [50]:
#13. Create a global temp view from a PySpark DataFrame named global_orders.
df.createOrReplaceGlobalTempView("global_orders")
#14. Run a SQL query on the global view from another notebook cell/session.
spark.sql("select * from global_temp.global_orders").show()

+---------+-----------------+-----------+-------+--------+----------+
|ProductID|      ProductName|   Category|  Price|Quantity|  SaleDate|
+---------+-----------------+-----------+-------+--------+----------+
|      206|           Laptop|Electronics|75000.0|       2|2024-06-06|
|      207|        Bookshelf|  Furniture| 5200.0|       1|2024-06-07|
|      208|          Pen Set| Stationery|  120.0|      15|2024-06-08|
|      209|Bluetooth Speaker|Electronics| 3200.0|       3|2024-06-09|
|      210|       Coffee Mug|Accessories|  350.0|       5|2024-06-10|
+---------+-----------------+-----------+-------+--------+----------+



In [51]:
#15. Create a second table customer_details with:
#CustomerID, Name, Gender, City, SignupDate
spark.sql("""create table if not exists customer_details(CustomerID INT,
Name STRING, Gender STRING, City STRING, SignupDate DATE) using parquet""")
#16. Insert at least 3 records into customer_details.
spark.sql("""
insert into customer_details values
  (201, 'Alice Sharma', 'Female', 'Mumbai', DATE '2024-05-12'),
  (202, 'Ravi Verma', 'Male', 'Delhi', DATE '2024-05-15'),
  (203, 'Priya Singh', 'Female', 'Bangalore', DATE '2024-05-20')
""")
#17. Write a SQL join between product_sales and customer_details based on
#ProductID = CustomerID (simulate a match).
spark.sql("""
select p.*, c.Name, c.Gender, c.City, c.SignupDate
from product_sales p
join customer_details c ON p.ProductID = c.CustomerID
""").show()

#18. List customers who bought more than 2 products.
spark.sql("""
select c.Name, SUM(p.Quantity) AS TotalQuantity
from customer_details c
join product_sales p ON c.CustomerID = p.ProductID
group by c.Name
having SUM(p.Quantity) > 2
""").show()


+---------+------------+-----------+-------+--------+----------+------------+------+---------+----------+
|productid| productname|   category|  price|quantity|  saledate|        Name|Gender|     City|SignupDate|
+---------+------------+-----------+-------+--------+----------+------------+------+---------+----------+
|      203|    Notebook| Stationery|   50.0|      10|2024-06-03| Priya Singh|Female|Bangalore|2024-05-20|
|      201|  Smartphone|Electronics|45000.0|       3|2024-06-01|Alice Sharma|Female|   Mumbai|2024-05-12|
|      202|Office Chair|  Furniture| 3000.0|       2|2024-06-02|  Ravi Verma|  Male|    Delhi|2024-05-15|
+---------+------------+-----------+-------+--------+----------+------------+------+---------+----------+

+------------+-------------+
|        Name|TotalQuantity|
+------------+-------------+
|Alice Sharma|            3|
| Priya Singh|           10|
+------------+-------------+



In [52]:
#19. Create a SQL view sales_summary that includes:
#ProductName, Price, Quantity, Total = Price * Quantity

spark.sql("""create or replace view sales_summary as
select productname, price, quantity, price * quantity as total
from product_sales
""")

#20. Query the view for records with
#Total > 1000 .
spark.sql("""
select *
from sales_summary
where total > 1000
""").show()


+------------+-------+--------+--------+
| productname|  price|quantity|   total|
+------------+-------+--------+--------+
|  Headphones| 1500.0|       4|  6000.0|
|Water Bottle|  250.0|       6|  1500.0|
|  Smartphone|45000.0|       3|135000.0|
|Office Chair| 3000.0|       2|  6000.0|
+------------+-------+--------+--------+



In [53]:
#21. Drop the view sales_summary .
spark.sql("""drop view if exists sales_summary""")

#22. Drop the tables product_sales and customer_details
spark.sql("drop table product_sales")
spark.sql("drop table customer_details")
#23. Drop the database sales_db .
spark.sql("drop database sales_db")

DataFrame[]